# Learning to play the game of Go without human input

In Lecture 9, we have learned about multi-agent systems and adverserial games. A central algorithm in game playing is **MiniMax**.

MiniMax essentially searches the entire space of moves possible by both players at any given time till the game reaches an end. This is how it maximizes reward while the opponent is trying to minimize it. We saw an optimization called alpha-beta pruning for MiniMax which is a great improvement but only by a factor of 2. This might help in games such as chess (an AI agent can be twice as smart on the same computer now assuming it uses a plausible heuristic).

TicTacToe has $3^9 = 19683$ possible states, roughly half of them are valid states. This kind of state space can be fully searched by computers.

Some games such as Go are way too complicated.

By [one estimate](https://www.google.com/search?q=2081681993819799846994786333448627702865224+5388453054842563945682092741961273801537852+5648451698519643907259916015628128546089888+314427129715319317557736620397247064840935&filter=0&biw=1280&bih=627) there are:

```
2081681993819799846994786333448627702865224
5388453054842563945682092741961273801537852
5648451698519643907259916015628128546089888
314427129715319317557736620397247064840935
```

States!

That certainly cannot be handled by even the biggest supercomputers or an earth filled with GPUs.


Go is a [googol](https://en.wikipedia.org/wiki/Googol) ($10^{100}$) times more complex than chess. It has more states than there are atoms in the universe. It has been a formidable challenge for AI.


In March 2016 DeepMind's AlphaGo Lee became the first program to beat a human expert -- 18-time world champion Lee Sedol. (DeepBlue beat Gary Kasparov in chess in 1997 -- super human ability in Go took 19 years longer).

The key insight of this software was to use a neural network to learn the game by looking at how humans play (using plays from ameteur players on internet Go servers) it to begin with, but later by playing the program against itself.

In 19 Oct 2017 DeepMind unweiled a newer, more elegant version of AlphaGo -- the AlphaGo Zero. This version learnt the game all on its own through playing millions of games with its best self. In 3 days, AlphaGo Zero trained enough to beat the first version (AlphaGo Lee) 100 games to 0! In this notebook we will look at the technique carried out in this "training". We also implment the algorithm on the game of Tictactoe and allow you to explore how the skill of the agent improves as it learns to play better.

## AlphaGo Zero's algorithm

### Background

#### Monte carlo tree search (MCTS)


![](https://nikcheerla.github.io/deeplearningschool//media/branching.jpg)

One way to non-exhaustively search a large state space graph is to **pick a random sample of branches**, playing each one many times to see which ones result in a victory down the road more often. This is called Monte-carlo tree search.

While playing a number of random games we can store the average reward for all plays so far from a given state. At a later time we can commit to using these probability as the "prior" for the random sampling of branches (i.e. higher the probability of winning, the more likely the branch will be sampled.)

Below is the simple Julia code which implements a simple form of MCTS for the game of tic-tac-toe

- the `randomplay` function runs a random game from beginning to end
- the `mcts_priors` runs randomplay `n` times and returns a dictionary with the average reward for each state it has explored thus far
- the `mcts_move` takes this dictionary, a state and a player and returns a next move (by picking the one that takes you to the state with the highest reward)

In [4]:
function randomplay(state, player)
    w = winner(state)
    if w == E && all(!iszero, state)
        return 0, [state] # draw
    elseif w != E
        return (w == X ? 1 : -1), [state]
    end
    succ = next_states(state, player)
    score, states = randomplay(rand(succ)[2], player == X ? O : X)
    return score, vcat([state], states)
end

function mcts_priors(n, f=randomplay)
    stats = Dict{State, Tuple{Int, Int}}() # we store sum and number of times chosen
    for i=1:n
        score, states = f(start_state, X) # run a random play -- gives score and a state
        # for each state we keep a sum of scores and the
        # number of times the state was taken
        for s in states[2:end] # leave out start state
            sum, count = haskey(stats, s) ? stats[s] : (0, 0)
            sum += score
            count += 1
            stats[s] = (sum, count)
        end
    end

    Dict(state => stat[1] / stat[2] for (state, stat) in stats)
end

function mcts_move(state, player, priors)
    succ = next_states(state, player)
    (move, p), idx = findmax([(s[1], get(priors, s[2], 0)) for s in succ])
    move
end


mcts_move (generic function with 1 method)

#### Reinforcement learning (specifically Q-learning)

An alternative to tree search reinforcement learning. In this technique, the agent learns to deal with an environment (like a game) by exploring it. The agent starts out acting randomly but observes what reward its action beget and over time figures out which actions result in good rewards.

A specific type of reinforcement learning is Q-learning. It consists of storing a table of rewards obtained by doing a certain action at a certain state. i.e. $Q : S \times A \rightarrow \mathbb{R}$

The **Q-learning** algorithm works as follows:

![](https://cdn-images-1.medium.com/max/1600/1*QeoQEqWYYPs1P8yUwyaJVQ.png)

![](https://cdn-images-1.medium.com/max/2000/0*voKUaGu68-cDuncy.)


Hence the Q function here needs to store $|S|\times|A|$ number of values in order to work.

Q-learning will work in training an agent, for example to play games like tic-tac-toe, the $Q$ table will be of size $3^9 = 19683$ rows. (each cell can have 3 states, one of the states is the action.)


#### An approximate Q-function

The game of Go has many more states, hence it is infeasible to create a full Q-table for the game. It seems we have circled back at the same problems as search. But there is a clever trick that may let us get away with storing much less data -- **approximate** the Q-table. A modern trendy approach to approximating non-linear functions like $Q$ is to train a neural network to approximate this.

More background about neural netwoks may be outside the scope of this document. Later on in the notebook we do not allude to any particular such details. It suffices to know that neural networks are black-boxes that approximate any function using a fixed amount of memory. And deep neural networks maybe thought of, approximately, as many such boxes chained together. You can however find some specifications of the type of neural networks we used down in the appendix.

![](https://cdn-images-1.medium.com/max/1600/1*yg9-6oLTAX0sVeLBJudexA.png)
_A neural network that approximates a Q function_

  - Start off with a random neural network
  - Keep playing the game to improve the $Q$ function it approximates
  
Reinforcement learning with deep neural networks (we will call this **Deep Q-learning**) have been used to play a [host of Atari games (pdf)](https://arxiv.org/pdf/1312.5602v1.pdf).
This is not exactly what AlphaGo zero uses, but it can be used to play a lot of games including Breakout.

Deep Q-learning is not exactly what is used by the AlphaGo player. It however incorporates two elements from the strategy of deep Q-learning.

1. Reinforcement learning (learning by actually playing)
2. Use of deep neural networks


**Approximating a function that guides MCTS**

AlphaGo zero creates a neural network $f_{\theta}$ which approximates the probability distribution among all branches for the next move. Then the sampling algorithm draws from this probability distribution. It also uses the same neural network to guess the expected probability of winning from the state it is applied to.


$$(\textbf{p}, v) = f_{\theta}(s)$$

Here $\textbf{p}$ maps every action available at state $s$ to a probability value that it should be sampled.


**Self-play**

A neural network needs to be told which are good moves and which are not in order to train itself to be better at the game. AlphaGo Zero keeps improving by playing against itself. Winning the game gets a reward of 1, losing gets a reward of -1.

This is the simple algorithm that runs all of AlphaGo Zero!

## Notes about implementation

For the purpose of this presentation we chose [AlphaGo.jl](https://github.com/tejank10/AlphaGo.jl) a Julia implimentation of AlphaGo Zero algorithm. The package not only implements the algorithm for Go, but also for the game of [Gomoku](https://en.wikipedia.org/wiki/Gomoku) which is a **generalization of Tic-tac-toe** played on the Go board. The program allows you to tweak the size of the board and the number of squares that need to be taken up consequitively to win the game. Hence Tic Tac Toe is a special case of Gomoku when the board is of size 3x3 and you need 3 colinear squares to win.

See appendix for more about the structure of the neural networks we tested.

##### Setting up Julia packages

The following cell should install all the dependencies to run this notebook. Ignore any build errors to do with CUDAnative and CUDAdrv packages, these are not required but are installed by default by the AlphaGo implementation.

You will have to restart the Jupyter notebook server and start this notebook again to get some of the UI packages to work (this will let you play Tic tac toe against agents of different skill levels :-) )

In [ ]:
using Pkg
Pkg.activate("env") # like virtualenv we have some package state files here
Pkg.resolve()       # installs all the dependencies

In [ ]:
# A function to load saved models

using AlphaGo, Flux
using BSON: @load

function load_model(str, n, env::AlphaGo.GameEnv)
  @load str*"/agz_base-$n.bson" bn
  @load str*"/agz_value-$n.bson" value
  @load str*"/agz_policy-$n.bson" policy

  @load str*"/weights/agz_base-$n.bson" bn_weights
  @load str*"/weights/agz_value-$n.bson" val_weights
  @load str*"/weights/agz_policy-$n.bson" pol_weights

  Flux.loadparams!(bn, bn_weights)
  Flux.loadparams!(value, val_weights)
  Flux.loadparams!(policy, pol_weights)

  NeuralNet(env; base_net=bn, value=value, policy=policy)
end

In [1]:
include("ui.jl") # The game-playing UI code

┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/WebIO/v9IED.ji for WebIO [0f1e0344-ec1d-5b48-a673-e5cf874b6c29]
└ @ Base loading.jl:1187


HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/e8f953a4abf5e4095b83e225fc5e11254819fa44-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/InteractBase/63c8J.ji for InteractBase [d3863d7c-f0c8-5437-a7b4-3ae773c01009]
└ @ Base loading.jl:1187
┌ Info: Recompiling stale cache file /home/shashi/.julia/compiled/v1.0/AlphaGo/hIRw1.ji for AlphaGo [da2c498e-f8db-5b64-9cc4-315cb361752b]
└ @ Base loading.jl:1187


LoadError: UndefVarError: AlphaGo not defined

### Playing against the neural network

Below is a UI where you can play against the neural network at various stages of its learning.

In the cell immediately below, the neural network has been only trained on 10 games.

The agent is smart to win or draw most games already, but try playing (1,1), (3,3), (1, 3), (1, 2) -- you will win this game. (note: $(i,j)$ is $i^{th}$ row $j^{th}$ column.)

In the subsequent cells you can adjust the level of training the Neural network has had and play the same game to see where the Neural network figured out to draw this game.

In [11]:

# play with the dumbest network first:
tictactoe = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe

# there are 50 levels we have trained, second argument is the level
nn = load_model("models_48_81boot", "1", tictactoe)


# play_with lets you play the game with any given neural network
b,t = play_with(tictactoe, nn)
b # -- this is the board, and gets interactively displayed

# NOTE: once you click on a square it may take several seconds for the first move to be registered
# Julia is a just-in-time compiled language, the first move compiles the whole
# program to C-like machine code, hence be patient

### Play with an agent which has various levels of training!

In [12]:
@manipulate for level = slider(1:50, label="Experience level (10 games each): ", value=1)
    gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
    nn = load_model("models_48_81boot", string(level), gametype) # there are 50 levels we have trained, second argument is the level

    play_with(gametype, nn)[1]
end

In [13]:
include("optimal.jl")

## Comparison against optimal player over time

The true test of skill for the neural network is comparison against the optimal policy for the game. Fortunately, in the game of Tic tac toe we can actually have do this. We have written an optimal player in `optimal.jl` file, it is in the same vein as the one in Homework 6. If you are curious about how the Julia code looks, you may have a look in there. We also have a function play_optimal which will play against a given neural network and return the result 1 if the optimal player wins 0 if a draw and -1 if the neural network wins (this never happens, since if all players played optimally there can only ever be a draw).

If you play every level of the NN player against the optimal player many times, you can plot what percentage of games a given level of player can draw or win, the plot hence shows the effectiveness of learning.

```julia
addprocs(4) # add 4 more Julia processors
using Distributed

stats = pmap(1:50) do i # run the evaluation in parallel
        gametype = AlphaGo.GomokuEnv(3,3) # the specific case which is tictactoe
        nn = load_model("models_48_81boot", string(i), gametype)

        plays = [play_optimal(gametype, nn) for i=1:20]
    end

end


using Gadfly
Gadfly.plot(y=(20 .- sum.(stats)) / 20 * 100, x=[1:50;] .* 10,
    Geom.point, Geom.smooth, Guide.xlabel("Games trained on "), Guide.ylabel("NN vs. Optimal (% not lost)"))
```

<img src="https://user-images.githubusercontent.com/25916/47750325-8f5f5f80-dc65-11e8-9a23-6dd0475d238a.png" width="600">

## Appendix

We initially trained the network on 10000 games without considering the option `start_training_after` which is by default set to 50000 -- these are the number of sample branches explored. So our training did not start to learn anything until it played about 6000 games, but was sampling games to assign priors for the monte carlo tree search. However, once the training started the priors accumulated from the 6000 games were enough to quickly learn the optimal strategy. The plot below shows this.


<img src="https://user-images.githubusercontent.com/25916/47750905-cc782180-dc66-11e8-8ac8-b38a4154f26b.png" width="600">

However for the purpose of the examples here we went with the much lesser trained network of a setup where we only use 81 initial games to set the prior for the monte-carlo sampling. Also we only trained on 500 games and captured a snaptshot of the agent more frequently. This makes for a much more human-accessible example. You can actually play against the agent and see how dumb it started out and how it learned specific moves.


### Don Knuth takes a crack at TicTacToe in 10000 bytes

Here is Donald Knuth talking about solving the game of tic tac toe by training against an optimal player https://www.youtube.com/watch?v=_c3dKYrjj2Q . The idea of learning from self-play in AlphaGo are certainly not new, but the cleverness is in the monte carlo tree search. And of course the enormous amount of compute power required to train the network.

## References

- **Mastering the game of Go without human knowledge**  -- Nature (Oct. 2017)
David Silver, Julian Schrittwieser, Karen Simonyan, Ioannis Antonoglou, Aja Huang, Arthur Guez, Thomas Hubert, Lucas Baker, Matthew Lai, Adrian Bolton, Yutian Chen, Timothy Lillicrap, Fan Hui, Laurent Sifre, George van den Driessche, Thore Graepel & Demis Hassabis

- **AlphaGo Zero Explained - On AI** blog post https://nikcheerla.github.io/deeplearningschool/2018/01/01/AlphaZero-Explained/

- **AlphaGo.jl** - Julia implementation of the AlphaGo Zero problem on Go and Gomoku -- a generalization of tic-tac-toe